In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns

In [3]:
data_train = pd.read_csv('data/train.csv')
data_test = pd.read_csv('data/test.csv')

In [4]:
data_train.sample(3)

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
9717,9718,3822,52,14,799,46,4265,226,209,113,...,0,0,0,0,0,0,0,0,1,7
2168,2169,2801,153,12,342,3,450,234,240,135,...,0,0,0,0,0,0,0,0,0,5
2096,2097,3320,104,15,170,37,2405,245,219,99,...,0,0,0,0,0,0,0,1,0,7


In [5]:
# we make x to train
x_all = data_train.drop(['Cover_Type'], axis =1)
# z to train
y_all = data_train['Cover_Type']

In [6]:
from sklearn.model_selection import train_test_split
num_test = 0.20
X_train, X_test, y_train, y_test = train_test_split(x_all, y_all,
                                                    test_size=num_test,
                                                    random_state=23)

In [7]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import make_scorer, accuracy_score

knn = KNeighborsClassifier(n_neighbors=7)

knn.fit(X_train, y_train);

In [8]:
pred = knn.predict(X_test)

In [9]:
print(accuracy_score(y_test, pred))

0.8048941798941799


In [10]:
data_test.sample(3)

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40
388242,403363,3228,222,6,509,73,2128,213,246,170,...,1,0,0,0,0,0,0,0,0,0
68669,83790,2945,4,10,454,30,3667,207,220,151,...,0,0,0,0,0,0,0,0,0,0
150931,166052,2768,106,14,60,-14,789,243,223,107,...,0,0,0,0,0,0,0,0,0,0


In [11]:
pred_final = knn.predict(data_test)
pred_final

array([1, 1, 1, ..., 3, 3, 3], dtype=int64)

In [13]:
raw_data = {'Id': data_test['Id'], 
            'Cover_Type': pred_final
           }
df = pd.DataFrame(raw_data, columns = ['Id', 'Cover_Type'])
df.to_csv('submission.csv',index=None)


In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier()

parameters = {'n_estimators': [4, 6, 9], 
              'max_features': ['log2', 'sqrt','auto'], 
              'criterion': ['entropy', 'gini'],
              'max_depth': [2, 3, 5, 10], 
              'min_samples_split': [2, 3, 5],
              'min_samples_leaf': [1,5,8]
             }
# metric to compare
acc_scorer = make_scorer(accuracy_score)
# run grid search
grid_obj = GridSearchCV(clf, parameters, scoring=acc_scorer)
grid_obj = grid_obj.fit(X_train, y_train)
# set the clf to the best combination of parameters
clf = grid_obj.best_estimator_
# fit best algorithm
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=10, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=5,
            min_weight_fraction_leaf=0.0, n_estimators=9, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [15]:
pred = clf.predict(X_test)
print(accuracy_score(y_test, pred))

0.7817460317460317


In [16]:
pred_final = clf.predict(data_test)


In [17]:
raw_data = {'Id': data_test['Id'], 
            'Cover_Type': pred_final
           }
df = pd.DataFrame(raw_data, columns = ['Id', 'Cover_Type'])
df.to_csv('submission.csv',index=None)